# Toronto Dwellings Analysis Dashboard

In this notebook, we compile the visualizations from the previous analysis to create a Panel dashboard.

In [15]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import panel.interact as interact
import os
from pathlib import Path
from dotenv import load_dotenv

In [16]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [17]:
# Read the Mapbox API key
load_dotenv()
mapbox_api_key = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(mapbox_api_key)

# Import Data

In [18]:
# Import the CSVs to Pandas DataFrames
file_path = Path("./data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("./data/toronto_neighbourhoods_coordinates.csv")
neighbourhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, functions are declared to manipulate data and create plots.

### Global available data

In [19]:
# Getting the data from the top 10 expensive neighbourhoods
def get_top_ten_expensive_neighbourhoods(data):
    
    most_expensive_neighbourhoods = (
        data[['neighbourhood', 'average_house_value']]
        .groupby(['neighbourhood'])
        .mean()
        .sort_values(by='average_house_value', ascending=False)
    )

    return most_expensive_neighbourhoods[:10]


# Calculate the mean number of dwelling types units per year
def get_mean_dwelling_types_per_year(data):

    dwelling_types = data.drop(columns=data.iloc[:,9:])

    mean_dwelling_types_per_year = (
        dwelling_types
        .groupby([dwelling_types.index, 'neighbourhood'])
        .mean()
        .astype(int)
    )

    return mean_dwelling_types_per_year

# Calculate the average monthly shelter costs for owned and rented dwellings
def get_average_monthly_shelter_costs_per_year(data):

    data.head()

    shelter_costs = (
        data[['shelter_costs_owned', 'shelter_costs_rented']]
        .groupby([to_data.index])
        .mean()
        .round(2)
    )

    return shelter_costs


def get_average_house_value_per_year(data):
    """
    Average house values per year.
    """
    house_values = (
        data[['average_house_value']]
        .groupby([data.index])
        .mean()
        .round(2)
    )

    return house_values
    

def get_average_house_value_by_neighbourhood(data):
    """
    Average house values by neighbourhood.
    """
    house_values = (
        to_data[['neighbourhood', 'average_house_value']]
        .groupby([to_data.index, 'neighbourhood'])
        .mean()
        .astype(int, copy=True)
    )

    return house_values

def get_average_house_value_snapshot(data):
    """
    Average house value for all Toronto's neighbourhoods per year.
    """
    house_value_snapshot = (
        get_average_house_value_by_neighbourhood(data)
    )

    house_value_snapshot.reset_index(inplace=True)

    house_value_snapshot.columns = [
        'Year', 
        'Neighbourhood', 
        'Avg. House Value'
    ]

    return house_value_snapshot
    

def get_number_dwelling_types(data):
    """
    Number of dwelling types per year
    """
    dwelling_types = (
        data
        .groupby([data.index])
        .sum()
    )

    dwelling_types.drop(columns=[
            'average_house_value', 
            'shelter_costs_owned', 
            'shelter_costs_rented'
        ], 
        inplace=True
    )

    return dwelling_types
    
    
def get_top_ten_expensive_neighbourhoods_yearly(data):
    """
    Top 10 most expensive neighbourhoods.
    """
    grouped_data = (
        data
        .groupby([to_data.index, 'neighbourhood'])
        .mean()
        .sort_values(by=['year', 'average_house_value'], ascending=False)
        .reset_index()
    )

    top_ten_2001 = grouped_data[grouped_data['year'] == 2001][:10]
    top_ten_2011 = grouped_data[grouped_data['year'] == 2011][:10]
    top_ten_2006 = grouped_data[grouped_data['year'] == 2006][:10]
    top_ten_2016 = grouped_data[grouped_data['year'] == 2016][:10]

    # Join the dataframes
    neighbourhoods = pd.concat(
        [top_ten_2001, top_ten_2006, top_ten_2011, top_ten_2016], 
        join='inner', 
        axis='rows'
    )

    return neighbourhoods

def get_merged_housing_data(data):
    """
    Get dataframe with merged location data and housing value data
    """
    neighbourhood_mean_values = (
        to_data
        .groupby(['neighbourhood'])
        .mean()
    )

    # Join the average values with the neighbourhood locations
    merged_housing_data = neighbourhood_locations.merge(
        neighbourhood_mean_values, 
        left_on='neighbourhood', 
        right_on='neighbourhood'
    )

    return merged_housing_data
    


### Panel Visualization Functions

In [28]:
# Define Panel visualization functions
def create_neighbourhood_map(data, title):
    """Neighbourhood Map"""
    average_house_value = get_merged_housing_data(data)

    scatter_plot = px.scatter_mapbox(
        average_house_value,
        lat='lat',
        lon='lon',
        color='average_house_value',
        size='average_house_value',
        width=1000,
        height=500,
        title=title,
        size_max=11,
        zoom=9.2,
        color_continuous_scale='Viridis',
        mapbox_style='carto-positron',
        hover_data={
            'neighbourhood': True,
            'lat': False,
            'lon': False
        }
    )

    return scatter_plot
    

def create_bar_chart(data, title, xlabel, ylabel, color):
    """
    Create a barplot based in the data argument.
    """
    bar_chart = data.hvplot.bar(
        xlabel=xlabel, 
        ylabel=ylabel, 
        color=color, 
        title=title,
        height=500,
        rot=90,
    )

    return bar_chart

def create_bar_chart_with_dropdown(data, title, xlabel, ylabel):

    dwelling_types = get_mean_dwelling_types_per_year(data)

    bar_chart = dwelling_types.hvplot.bar(
        x='year',
        groupby='neighbourhood',
        xlabel=xlabel, 
        ylabel=ylabel,
        title=title,
        height=500,
        rot=90,
    )

    return bar_chart

def create_bar_chart_row_facet(data, x, y, title):
    """
    Create a barplot based in the data argument.
    """
    rows = get_average_house_value_by_neighbourhood(data)
    rows.reset_index(inplace=True)
    rows.columns = ['Year', 'Neighbourhood', 'Avg. House Value']

    bar_chart_row_facet = px.bar(
        rows,
        x=x, 
        y=y, 
        color='Avg. House Value', 
        title=title,
        height=700,
        facet_row='Year',
    )

    return bar_chart_row_facet
    

def create_line_chart(data, title, xlabel, ylabel, color):
    """
    Create a line chart based in the data argument.
    """
    line_chart = data.hvplot.line(
        xlabel=xlabel, 
        ylabel=ylabel, 
        color=color, 
        title=title, 
        figsize=(15,5),
    )

    return line_chart


def create_line_chart_with_dropdown(data, title, xlabel, ylabel, color):
    
    line_chart = data.hvplot.line(
        x='year', 
        y='average_house_value', 
        groupby='neighbourhood', 
        color=color,
        xlabel=xlabel,
        ylabel=ylabel,
        title=title,
    )

    return line_chart



def create_sunburst_chart(data, title):
    """
    Sunburst chart to conduct a costs analysis of most expensive
    neighbourhoods in Toronto per year.
    """

    chart_data = get_top_ten_expensive_neighbourhoods_yearly(data)

    sunburst_chart = px.sunburst(
        chart_data, 
        path=['year', 'neighbourhood'], 
        values='average_house_value', 
        color='average_house_value', 
        color_continuous_scale='blues',
        title=title
    )

    return sunburst_chart
    

## Panel Dashboard

Finally, all plots are combined into a single dashboard view using Panel.

### Create the Welcome view

In [21]:
# Create a Title for the Dashboard
dashboard_title = pn.Column(
    '## Real Estate Analysis of Toronto from 2001 to 2016'
)

# Define a welcome text
welcome_message = '''
Welcome to Toronto's Real Estate Analysis dashboard. In this dashboard 
you will find historical house values, dwelling types by neghbourhood and 
their associated costs over the years. Navigate through the tabs above to 
see more details on the evolution of Toronto's real estate market."
'''

# Create the Welcome view
scatter_plot = create_neighbourhood_map(
    to_data, 
    title="Average House Values in Toronto"
)

welcome_pane = pn.Column(
    welcome_message,
    scatter_plot,
)

### Create the Yearly Market Analysis view

In [22]:

# Get the mean number of dwelling types per year
dwelling_types_per_year = get_number_dwelling_types(to_data)

# Plot data for each year
bar_chart_2001 = create_bar_chart(
    dwelling_types_per_year.loc[2001], 
    'Dwelling Types in Toronto in 2001', 
    '2001', 
    'Dwelling Type Units', 
    'DodgerBlue',
)

bar_chart_2006 = create_bar_chart(
    dwelling_types_per_year.loc[2006], 
    'Dwelling Types in Toronto in 2006', 
    '2006', 
    'Dwelling Type Units', 
    'DeepPink',
)

bar_chart_2011 = create_bar_chart(
    dwelling_types_per_year.loc[2011], 
    'Dwelling Types in Toronto in 2006', 
    '2011', 
    'Dwelling Type Units', 
    'LimeGreen',
)
bar_chart_2016 = create_bar_chart(
    dwelling_types_per_year.loc[2016], 
    'Dwelling Types in Toronto in 2016', 
    '2016', 
    'Dwelling Type Units', 
    'DarkViolet',
)

# Create the Yearly Market Analysis view
yearly_market_analysis_pane = pn.Column(
    pn.Row(bar_chart_2001, bar_chart_2006),
    pn.Row(bar_chart_2011, bar_chart_2016),
)

### Create the Shelter Costs vs. House Value view

In [23]:
monthly_shelter_costs = get_average_monthly_shelter_costs_per_year(to_data)

shelter_costs_owned_chart = create_line_chart(
    monthly_shelter_costs[['shelter_costs_owned']], 
    'Average Monthly Shelter Cost for Owned Dwellings in Toronto', 
    'Year', 
    'average. Monthly Shelter Cost', 
    'DodgerBlue'
)

# Line chart for rented dwellings
shelter_costs_rented_chart = create_line_chart(
    monthly_shelter_costs[['shelter_costs_rented']], 
    'Average Monthly Shelter Cost for Rented Dwellings in Toronto', 
    'Year', 
    'average. Monthly Shelter Cost', 
    'DeepPink'
)

average_house_value_per_year = get_average_house_value_per_year(to_data)

average_house_value_chart = create_line_chart(
    average_house_value_per_year, 
    'Average House Value in Toronto', 
    'Year', 
    'Avg. House Value', 
    'DodgerBlue'
)

shelter_costs_pane = pn.Column(
    shelter_costs_owned_chart,
    shelter_costs_rented_chart,
    average_house_value_chart
)

### Create the Neighbourhood Analysis view

In [24]:

average_house_values = get_average_house_value_by_neighbourhood(to_data)

average_house_values_line_chart = create_line_chart_with_dropdown(
    average_house_values,
    title="Average House Value by Neighbourhood",
    xlabel='Avg. House Value',
    ylabel='Year',
    color='DodgerBlue',
)

dwelling_types_bar_chart = create_bar_chart_with_dropdown(
    to_data,
    title="Dwelling Type Units per Year",
    xlabel='Dwelling Type Units',
    ylabel='Year',

)

average_values_bar_chart = create_bar_chart_row_facet(
    to_data,
    x='Neighbourhood',
    y='Avg. House Value',
    title="Average House Values in Toronto by Neighbourhood",
)

left_column = pn.Column(
    average_house_values_line_chart,
    dwelling_types_bar_chart,
)

right_column = pn.Column(
    average_values_bar_chart 
)

neighbourhood_analysis_pane = pn.Row(
    left_column,
    right_column,
)

### Create the Top Expensive Neighbourhoods column

In [25]:
top_ten = get_top_ten_expensive_neighbourhoods(to_data)
top_ten_bar_chart = create_bar_chart(
    top_ten,
    title="Top 10 Expensive Neighbourhoods in Toronto",
    xlabel='Neighbourhood',
    ylabel='Avg. House Value',
    color='DodgerBlue'
)

top_ten_yearly_sunburst_chart = create_sunburst_chart(
    to_data,
    title="Cost Analysis of Most Expensive Neighbourhoods in Toronto per Year"
)

top_ten_pane = pn.Row(
    top_ten_bar_chart,
    top_ten_yearly_sunburst_chart,
)


### Group panes into tabs

In [26]:
tabs = pn.Tabs(
    ("Welcome", welcome_pane),
    ("Yearly Market Analysis", yearly_market_analysis_pane),
    ("Shelter Costs vs. House Value", shelter_costs_pane),
    ("Neighbourhood Analysis", neighbourhood_analysis_pane),
    ("Top Expensive Neighbourhoods", top_ten_pane),
)

# Create a tab layout for the dashboard
dashboard = pn.Column(
    "## Toronto Real Estate Analysis from 2001 to 2016",
    tabs
)


## Serve the Panel Dashboard

In [27]:
dashboard.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'2a38a9b1-defc-4bc7-b83e-c312dd57cdb5': {'defs': …

Column
    [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Markdown(str)
            [1] Plotly(Figure)
        [1] Column
            [0] Row
                [0] HoloViews(Bars)
                [1] HoloViews(Bars)
            [1] Row
                [0] HoloViews(Bars)
                [1] HoloViews(Bars)
        [2] Column
            [0] HoloViews(Curve)
            [1] HoloViews(Curve)
            [2] HoloViews(Curve)
        [3] Row
            [0] Column
                [0] Row
                    [0] HoloViews(DynamicMap)
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                        [1] VSpacer()
                [1] Row
                    [0] HoloViews(DynamicMap)
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                        [1] VSpacer()
            [1] Column
                [0] Plotly(Figure)
        [4] Row
            [0] HoloViews(Bars)
            [1] Plotly(Figure)